In [1]:
# setup
import pandas as pd
from chronos import Chronos2Pipeline
import matplotlib.pyplot as plt  
import os

c:\Users\lexei\tf-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
context_df = pd.read_csv("telemetry_anonymized.csv", parse_dates=["Last Seen"])

context_df = context_df.sort_values(["host_name (anonymized)", "Last Seen"])

# Count rows per series
series_lengths = context_df.groupby("host_name (anonymized)").size()

print(series_lengths)
# Keep only series with >= 3 rows
valid_ids = series_lengths[series_lengths >= 3].index
context_df = context_df[context_df["host_name (anonymized)"].isin(valid_ids)]

print(context_df.head())


host_name (anonymized)
1025    1
1046    1
1049    1
1056    1
1084    1
       ..
9945    1
9950    1
9972    1
9984    1
9991    1
Length: 1000, dtype: int64
Empty DataFrame
Columns: [Event Id, Last Seen, host_name (anonymized), model_name, hardware_make, auth_username (anonymized), group_id, tenant_id, platform, metric_category, measure_name, time, p90_processor_time, avg_processor_time, max_cpu_usage, p90_memory_utilization, avg_memory_utilization, max_memory_usage, p10_battery_health, avg_battery_health, cpu_count, memory_count, memory_size_gb, driver_vendor, os, driver_version, driver_date, os_version, driver, agent_id, performance_status, device_status, max_battery_temperature, avg_battery_temperature, p90_battery_temperature, avg_cpu_temp, p90_cpu_temp, avg_battery_discharge, p90_battery_discharge, avg_boot_time, p90_boot_time, uptime_days, total_app_crash]
Index: []

[0 rows x 43 columns]


In [4]:
pipeline = Chronos2Pipeline.from_pretrained("amazon/chronos-2", device_map="cpu") # changed to cpu since I don't have a gpu

# Load historical target values and past values of covariates
#context_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/train.parquet")

# (Optional) Load future values of covariates
#test_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/test.parquet")
#future_df = test_df.drop(columns="target")

# Generate predictions with covariates
pred_df = pipeline.predict_df(
    context_df,
    future_df = None,
    prediction_length=24,  # Number of steps to forecast
    quantile_levels=[0.1, 0.5, 0.9],  # Quantiles for probabilistic forecast
    id_column="Event Id",  # Column identifying different time series
    timestamp_column="Last Seen",  # Column with datetime information
    target="host_name (anonymized)",  # Column(s) with time series values to predict
)

IndexError: list index out of range